In [16]:
import numpy as np
n = 1000
np.random.seed(123)
X_0 = np.full(n, 1)
X_1 = np.random.normal(2, 5, n)
X_2 = np.random.normal(5, 2, n)
X_3 = np.random.normal(5, 5, n)

In [17]:
np.random.seed(123)
x_0 = np.full(n, 1)
x_1 = np.random.normal(4, 3, n)
x_2 = np.random.normal(3, 2, n)
x_3 = np.random.normal(2, 1, n)

In [18]:
u = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=n)

In [19]:
Betas = np.array([[0, 1, -3, 1]])
betas = np.array([[1, 4, 2, -1]])
X = np.vstack((X_0, X_1, X_2, X_3)).T
x = np.vstack((x_0, x_1, x_2, x_3)).T

In [20]:
Y = (X * Betas).sum(axis=1) + u[:, 0]
y = (x * betas).sum(axis=1) + u[:, 1]

In [21]:
import pystan

Проверим работу модели на одной итерации.

In [22]:
%%time

data = {'m': 2, 'k': X.shape[1], 'n': n, 'x': x, 'X':X, 'y': np.array([y, Y]).T} 

model = pystan.stan(file = '/Users/efremovdanil/Desktop/Bayes/2_regressions_normal.stan', data = data, chains = 1,
                    iter = 2000, seed=123)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f316fa1c3ed00bf739ab06cfa1957bd6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


CPU times: user 3min 54s, sys: 2.63 s, total: 3min 56s
Wall time: 5min 6s


Значения коэффициентов получились похожими на истинные. Значения параметров ковариационной матрицы получились недостаточно похожими на истинные.

In [23]:
real = np.array([1, 4, 2, -1, 0, 1, -3, 1, 1, 0.5, 1])
results = np.array([model.extract()['beta'][:, 0].mean(), model.extract()['beta'][:, 1].mean(),
                     model.extract()['beta'][:, 2].mean(), model.extract()['beta'][:, 3].mean(), 
                     model.extract()['Beta'][:, 0].mean(), model.extract()['Beta'][:, 1].mean(), 
                     model.extract()['Beta'][:, 2].mean(), model.extract()['Beta'][:, 3].mean(), 
                     model.extract()['Sigma'][:, 1, 1].mean(), model.extract()['Sigma'][:, 1, 0].mean(), 
                     model.extract()['Sigma'][:, 0, 0].mean()])

In [24]:
import pandas as pd
df = pd.DataFrame(np.array([real, results]).T, columns=['Истинные значения', 'Получившиеся значения'])
df.index=['Beta_0', 'Beta_1', 'Beta_2', 'Beta_3', 'beta_0', 'beta_1', 'beta_2', 'beta_3', 'sigma_1', 
          'cov(u_1, u_2)', 'sigma_2']
df

,Истинные значения,Получившиеся значения
Beta_0,1.0,0.825309
Beta_1,4.0,4.004573
Beta_2,2.0,2.023178
Beta_3,-1.0,-0.944762
beta_0,0.0,-0.109215
beta_1,1.0,0.996416
beta_2,-3.0,-2.997280
beta_3,1.0,1.005416
sigma_1,1.0,2.011638
"cov(u_1, u_2)",0.5,1.451778
